# Convolutional Neural Network

### Libraries

In [ ]:
# import libraries

import os
import cv2
import numpy as np

from google.colab import drive
from xml.etree import ElementTree as ET

In [ ]:
# major variables

photos_dir = '/content/data/photos'
renders_dir = '/content/data/renders'

### Dataset

In [ ]:
# mount drive on colab notebook

drive.mount('/content/drive')

In [ ]:
# unzip data files

!unzip "/content/drive/MyDrive/02 - tagged1.zip" -d "/content/data"

In [ ]:
def parse_xml(xml_file):
    '''
    Read the xml file and return the bounding box coordinates
    '''
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bounding_boxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])
    return bounding_boxes

In [ ]:
def load_data(data_dir):
    '''
    Returns a list of images and labels for each image
    '''
    images = []
    labels = []
    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):
                image_path = os.path.join(subdir, file)
                image = cv2.imread(image_path)
                # Preprocess image (e.g., resize, normalize)
                # Add preprocessed image to images list
                images.append(image)
                label_path = os.path.splitext(image_path)[0] + '.xml'
                bounding_boxes = parse_xml(label_path)
                labels.append(bounding_boxes)
    return np.array(images), np.array(labels)

In [ ]:
# load data

photos_images, photos_labels = load_data(photos_dir)
renders_images, renders_labels = load_data(renders_dir)

In [ ]:
print("Photos Images Shape:", photos_images.shape)
print("Photos Labels Shape:", photos_labels.shape)
print("Renders Images Shape:", renders_images.shape)
print("Renders Labels Shape:", renders_labels.shape)